# Neural style transfer
As part of Fast AI's challenge for students, I will be implementing neural style transfer. Given that I am training primarily on my M1 MacBook Pro's GPU (read: slow training), I may set some aspects to be easier to train (e.g. less epochs, less classes etc).

Let's have a go at it and see what happens.

In [66]:
import shutil,os,torch,random,datasets,math
import fastcore.all as fc, numpy as np, matplotlib as mpl, matplotlib.pyplot as plt
import torchvision.transforms as T
import torchvision.transforms.functional as TF,torch.nn.functional as F

from torch.utils.data import DataLoader,default_collate
from pathlib import Path
from torch.nn import init
from fastcore.foundation import L
from torch import nn,tensor
from operator import itemgetter
from torcheval.metrics import MulticlassAccuracy
from functools import partial
from torch.optim import lr_scheduler
from torch import optim
from torchvision.io import read_image,ImageReadMode
from glob import glob
from sklearn.model_selection import train_test_split

from miniai.datasets import *
from miniai.conv import *
from miniai.learner import *
from miniai.activations import *
from miniai.init import *
from miniai.sgd import *
from miniai.resnet import *
from miniai.augment import *
from miniai.accel import *
from miniai.training import *

In [67]:
from fastprogress import progress_bar

In [68]:
torch.set_printoptions(precision=5, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['figure.dpi'] = 70

set_seed(42)
if fc.defaults.cpus>8: fc.defaults.cpus=8

## Data processing

In [69]:
path_data = Path('data')
path_data.mkdir(exist_ok=True)
path = path_data/'tiny-best-artwork'

We need to split our data into train test folders.

In [ ]:
data_prepped = True

In [75]:
if not data_prepped:
    for artist_folder in path.iterdir():
        if artist_folder.is_dir():
            files = list(glob(str(artist_folder/'*.jpg')))
            train_files, test_files = train_test_split(files, test_size=0.2)

            # Create `train` and `test` folders
            train_folder = artist_folder / 'train'
            test_folder = artist_folder / 'test'
            os.makedirs(train_folder, exist_ok=True)
            os.makedirs(test_folder, exist_ok=True)

            # Move files to `train` folder
            for train_file in train_files:
                shutil.move(train_file, train_folder / os.path.basename(train_file))

            # Move files to `test` folder
            for test_file in test_files:
                shutil.move(test_file, test_folder / os.path.basename(test_file))


In [70]:
bs = 64

In [98]:
class TinyDS:
    def __init__(self, path, train_or_test):
        self.path = Path(path)
        self.files = glob(str(self.path/f"*/{train_or_test}/*.jpg"), recursive=True)

    def __len__(self): return len(self.files)

    def __getitem__(self, i):
        return self.files[i], Path(self.files[i]).parent.parent.name

In [99]:
train_ds = TinyDS(path, "train")

In [100]:
len(train_ds)

1108

In [101]:
train_ds[0]

('data/tiny-best-artwork/Albrecht_Du╠êrer/train/Albrecht_Du╠êrer_14.jpg',
 'Albrecht_Du╠êrer')

In [102]:
val_ds = TinyDS(path, "test")
len(val_ds)
val_ds[0]

('data/tiny-best-artwork/Albrecht_Du╠êrer/test/Albrecht_Du╠êrer_247.jpg',
 'Albrecht_Du╠êrer')